# Data Preparation

## Packages

In [1]:
import os
import winsound
import numpy as np
import pandas as pd
import datetime as dt
from tqdm import tqdm
tqdm.pandas()
from multiprocessing import  Pool
import time
import imblearn

import math, collections
from scipy.stats import linregress

from matplotlib import pyplot as plt
from matplotlib.pyplot import figure

import warnings
warnings.filterwarnings('ignore')

In [2]:
def get_slope(y_axis):
    global data
    ma_len = len(y_axis)
    
    x_axis = []
    for i in range(ma_len):
        x_axis.append(1 + ((i+1) * 0.0001 * 0.1))
    
    slope_tick, intercept, _, _, _ = linregress(x_axis, y_axis)
    slope_tick = math.degrees(math.atan(slope_tick))
    
    return(slope_tick)

In [3]:
def chunk_ticks(df, number_of_ticks):   
    global data
    
    df['tick'] = (df['Bid'] + df['Ask'])/2
    df['spread'] = df['Ask'] - df['Bid']
    df = df[['tick', 'spread']]
    
    temp_df = pd.DataFrame()
    tick_avg = []
    spread_avg = []
    tick_sd = []
    tick_act = []
    
    for i in tqdm(range(0,len(df),number_of_ticks)):
        tick_list = list(df['tick'][i:i+number_of_ticks])
        spread_list = list(df['spread'][i:i+number_of_ticks])
        #print(len(tick_list))
        tick_act.append(tick_list[-1])              
        temp = list(pd.DataFrame(tick_list).ewm(span=len(tick_list)).mean()[0])[len(tick_list) - 1]
        tick_avg.append(temp)

        spread_avg.append(np.mean(spread_list))
        tick_sd.append(np.std(tick_list))
        
    temp_df['tick_act'] = tick_act      
    temp_df['tick_avg'] = tick_avg  
    temp_df['spread_avg'] = spread_avg  
    temp_df['tick_sd'] = tick_sd  
    
    return(temp_df)

In [4]:
def before_sma():
    global data    
    data['ssma_list'].append(val)    
    return()

def after_sma():
    global data
    
    data['ssma_list'].popleft()
    data['ssma_list'].append(val)
    data['sema'] = list(pd.DataFrame(list(data['ssma_list'])).ewm(span=data['sma_len']).mean()[0])[data['sma_len'] - 1]
    
    if len(data['sema_ready']) < 2:
        data['sema_ready'].append(data['sema'])
        data['sema_diff'] = np.nan

    elif len(data['sema_ready']) > 1:
        data['sema_ready'].popleft()
        data['sema_ready'].append(data['sema'])
        data['sema_diff'] = data['sema_ready'][-1] - data['sema_ready'][len(data['sema_ready'])-2]
    
    return()

def before_lma():
    global data    
    data['lsma_list'].append(val)    
    return()

def after_lma():
    global data
    
    data['lsma_list'].popleft()
    data['lsma_list'].append(val)
    data['lema'] = list(pd.DataFrame(list(data['lsma_list'])).ewm(span=data['lma_len']).mean()[0])[data['lma_len'] - 1]
    
    if len(data['lema_ready']) < 2:
        data['lema_ready'].append(data['lema'])
        data['lema_diff'] = np.nan

    elif len(data['lema_ready']) > 1:
        data['lema_ready'].popleft()
        data['lema_ready'].append(data['lema'])
        data['lema_diff'] = data['lema_ready'][-1] - data['lema_ready'][len(data['lema_ready'])-2]
    
    return()

In [5]:
def roll_ma(ma_list):
    global data
    ma_len = len(ma_list)
    sema_val = list(pd.DataFrame(ma_list).ewm(span=ma_len).mean()[0])[ma_len - 1]    
    return(sema_val)

## File paths

#### 1. High number of ticks ensures pip level movement and pip level predictions
#### 2. High sma_len helps ml model prediction accuracy (Predictable curve)
#### 3. High sma_len remove connection between actual tick_avg and sema (Prediction is high, but actual prediction is poor)

In [6]:
year = 2019

data = {}
data['number_of_ticks'] = 300
data['rsi_window'] = 14
data['sma_len'] = 5
data['lma_len'] = 10

diff_col = 'sema'
#diff_col = 'tick_avg'

data['pip_diff'] = 0.0001

source_file_path = f'data/yearly_tick_data/{year}.csv'
path, file_name = os.path.split(source_file_path)

target_file_name = 'tab_'+file_name
target_file_path = os.path.join(path, target_file_name)

chunk_file_name = 'chunk_'+file_name
chunk_file_path = os.path.join(path, chunk_file_name)

print(f'source_file_path : {source_file_path}')
print(f'chunk_file_path : {chunk_file_path}')
print(f'target_file_path : {target_file_path}')

source_file_path : data/yearly_tick_data/2019.csv
chunk_file_path : data/yearly_tick_data\chunk_2019.csv
target_file_path : data/yearly_tick_data\tab_2019.csv


## Read data

In [7]:
%%time
#df = pd.read_csv(source_file_path, nrows=10000000)
df = pd.read_csv(source_file_path)
df.head()

Wall time: 15.6 s


,DateTime,Bid,Ask,Volume
0,20190101 22:02:37.254,1.14598,1.14682,4
1,20190101 22:02:38.590,1.14599,1.14682,2
2,20190101 22:02:39.138,1.14599,1.14684,4
3,20190101 22:02:55.787,1.14598,1.14684,4
4,20190101 22:03:02.060,1.14598,1.14684,4


## Data manipulation

In [8]:
%%time

df = chunk_ticks(df, data['number_of_ticks'])
df.to_csv(chunk_file_path, index = False)
print(f'Records : {len(df)}')

df = pd.read_csv(chunk_file_path)
df.head()

100%|███████████████████████████████████████████████████████████████████████████| 97288/97288 [02:02<00:00, 796.68it/s]


Records : 97288
Wall time: 2min 4s


,tick_act,tick_avg,spread_avg,tick_sd
0,1.146260,1.146420,0.000481,0.000160
1,1.146320,1.146321,0.000478,0.000066
2,1.146325,1.146313,0.000327,0.000062
3,1.146315,1.146308,0.000254,0.000030
4,1.146530,1.146370,0.000418,0.000063


In [9]:
%%time

data['rs_max'] = 1e6

data['ssma_list'] = collections.deque([])
data['lsma_list'] = collections.deque([])
data['sema_ready'] = collections.deque([])
data['lema_ready'] = collections.deque([])
df['sema'] = ''
df['lema'] = ''
df['sema_diff'] = ''
df['lema_diff'] = ''


# RSI -----------------------------
df['diff'] = df['tick_avg'].diff()
df['gain'] = 0
df['loss'] = 0
df['gain'].loc[df['diff'] > 0] = abs(df['diff'])
df['loss'].loc[df['diff'] < 0] = abs(df['diff'])
df['avg_gain'] = df['gain'].rolling(window=data['rsi_window']).mean()
df['avg_loss'] = df['loss'].rolling(window=data['rsi_window']).mean()
df['rs'] = df['avg_gain']/df['avg_loss']
df['rs'] = df['rs'].where(df['rs'] <= data['rs_max'], data['rs_max']) 
df['rsi'] = 100 - (100 / (df['rs'] + 1))

# Simple Moving Averages ------------------
df['ssma'] = df['tick_avg'].rolling(window=data['sma_len']).mean()
df['ssma_diff'] = df['ssma'].diff()
df['lsma'] = df['tick_avg'].rolling(window=data['lma_len']).mean()
df['lsma_diff'] = df['lsma'].diff()
df['sma_diff'] = df['ssma'] - df['lsma']

df['max_tick'] = df['tick_avg'].rolling(window=data['sma_len']).max()
df['min_tick'] = df['tick_avg'].rolling(window=data['sma_len']).min()

df['max_gap'] = df['max_tick'] -  df['tick_avg']
df['min_gap'] = df['min_tick'] - df['tick_avg']

Wall time: 75.6 ms


In [10]:
%%time
# Emas ----------------
df['sema'] = df['tick_avg'].rolling(window=data['sma_len']).progress_apply(roll_ma)
df['lema'] = df['tick_avg'].rolling(window=data['lma_len']).progress_apply(roll_ma)

df['sema_diff'] = df['sema'].diff()
df['lema_diff'] = df['lema'].diff()

df['ema_diff'] = df['sema'] - df['lema']

97284it [01:30, 1075.38it/s]
97279it [01:29, 1081.29it/s]

Wall time: 3min


In [11]:
%%time
# Slopes -----------------------------
df['small_sema_slope'] = df['sema'].rolling(window=data['sma_len']).progress_apply(get_slope)
df['long_sema_slope'] = df['sema'].rolling(window=data['lma_len']).progress_apply(get_slope)

df['slope_diff'] = df['small_sema_slope'] - df['long_sema_slope']

97280it [00:27, 3563.14it/s]
97275it [00:28, 3430.43it/s]


Wall time: 55.7 s


In [12]:
%%time
df = df.round(5)

# Direction -------------------------
df['direction'] = 'same'
df['direction'].loc[df[diff_col].shift(-1) - df[diff_col] > data['pip_diff']] = 'increase'
df['direction'].loc[df[diff_col].shift(-1) - df[diff_col] < -data['pip_diff']] = 'decrease'

# Remove NaNs ------------------------
del df['gain']
del df['loss']
del df['tick_act']
df = df.dropna()
df = df.reset_index(drop=True)
print(f'Total records : {len(df)}')

df.tail()

Total records : 97275
Wall time: 220 ms


,tick_avg,spread_avg,tick_sd,sema,lema,sema_diff,lema_diff,diff,avg_gain,avg_loss,...,sma_diff,max_tick,min_tick,max_gap,min_gap,ema_diff,small_sema_slope,long_sema_slope,slope_diff,direction
97270,1.12169,0.00003,0.00006,1.12166,1.12174,0.00001,-0.00003,0.00010,0.00002,0.00008,...,-0.00015,1.12175,1.12159,0.00005,-0.0001,-0.00008,-76.69861,-82.21915,5.52054,same
97271,1.12156,0.00004,0.00011,1.12162,1.12169,-0.00004,-0.00005,-0.00013,0.00002,0.00008,...,-0.00013,1.12170,1.12156,0.00014,0.0000,-0.00007,-70.78843,-81.08444,10.29601,same
97272,1.12146,0.00003,0.00003,1.12156,1.12163,-0.00006,-0.00006,-0.00010,0.00002,0.00008,...,-0.00011,1.12170,1.12146,0.00024,0.0000,-0.00007,-73.62797,-80.03448,6.40651,same
97273,1.12140,0.00005,0.00006,1.12149,1.12157,-0.00007,-0.00006,-0.00006,0.00001,0.00009,...,-0.00011,1.12169,1.12140,0.00029,0.0000,-0.00008,-77.05995,-79.00590,1.94595,decrease
97274,1.12121,0.00010,0.00009,1.12138,1.12149,-0.00011,-0.00009,-0.00019,0.00001,0.00009,...,-0.00011,1.12169,1.12121,0.00048,0.0000,-0.00011,-81.87528,-78.80857,-3.06671,same


## Write data to csv

In [13]:
%%time
df.to_csv(target_file_path, index = False)
winsound.PlaySound('C:\\Windows\\Media\\tada.wav', winsound.SND_ASYNC)

Wall time: 1.96 s


## Print Report

In [14]:
g= df['direction']
print(pd.concat([g.value_counts(), g.value_counts(normalize=True).mul(100)],axis=1, keys=('counts','percentage')))
df.head(5)

          counts  percentage
same       78294   80.487278
decrease    9760   10.033410
increase    9221    9.479311


,tick_avg,spread_avg,tick_sd,sema,lema,sema_diff,lema_diff,diff,avg_gain,avg_loss,...,sma_diff,max_tick,min_tick,max_gap,min_gap,ema_diff,small_sema_slope,long_sema_slope,slope_diff,direction
0,1.14666,0.00007,0.00003,1.14664,1.14658,0.00001,0.00003,-0.00001,0.00005,0.00003,...,0.00012,1.14667,1.14658,0.00001,-0.00008,0.00006,70.61279,77.80651,-7.19371,same
1,1.14671,0.00006,0.00005,1.14667,1.14661,0.00002,0.00003,0.00005,0.00005,0.00003,...,0.00010,1.14671,1.14658,0.00000,-0.00013,0.00006,66.98876,78.39118,-11.40241,same
2,1.14668,0.00006,0.00002,1.14668,1.14664,0.00001,0.00003,-0.00003,0.00005,0.00003,...,0.00007,1.14671,1.14662,0.00003,-0.00005,0.00004,61.43258,76.35751,-14.92494,same
3,1.14660,0.00005,0.00002,1.14665,1.14664,-0.00003,-0.00000,-0.00008,0.00005,0.00003,...,0.00003,1.14671,1.14660,0.00011,0.00000,0.00001,34.36016,71.15660,-36.79644,same
4,1.14650,0.00005,0.00005,1.14659,1.14661,-0.00006,-0.00003,-0.00010,0.00005,0.00004,...,0.00000,1.14671,1.14650,0.00021,0.00000,-0.00002,-51.53254,58.29498,-109.82752,decrease


In [15]:
# Direction -------------------------
#diff_col = 'tick_act'
#data['pip_diff'] = 0.0001
diff_col = 'tick_avg'

df['act_direction'] = 'same'
df['act_direction'].loc[df[diff_col].shift(-1) - df[diff_col] > data['pip_diff']] = 'increase'
df['act_direction'].loc[df[diff_col].shift(-1) - df[diff_col] < -data['pip_diff']] = 'decrease'

print('prediction : same')
print(df.loc[df['direction'] == 'same', 'act_direction'].value_counts(normalize=True))
print('-------------')

print('prediction : increase')
print(df.loc[df['direction'] == 'increase', 'act_direction'].value_counts(normalize=True))
print('-------------')

print('prediction : decrease')
print(df.loc[df['direction'] == 'decrease', 'act_direction'].value_counts(normalize=True))
print('-------------')

df['tick_avg_diff'] = round(df['tick_avg'].diff() * 10000)

print('\n')
print(df[['tick_avg_diff','tick_avg', 'sema', 'direction','act_direction']].head(50))
del df['act_direction']
del df['tick_avg_diff']

prediction : same
same        0.643727
increase    0.180665
decrease    0.175607
Name: act_direction, dtype: float64
-------------
prediction : increase
increase    0.789177
same        0.196725
decrease    0.014098
Name: act_direction, dtype: float64
-------------
prediction : decrease
decrease    0.789549
same        0.195184
increase    0.015266
Name: act_direction, dtype: float64
-------------


    tick_avg_diff  tick_avg     sema direction act_direction
0             NaN   1.14666  1.14664      same          same
1             0.0   1.14671  1.14667      same          same
2            -0.0   1.14668  1.14668      same          same
3            -1.0   1.14660  1.14665      same          same
4            -1.0   1.14650  1.14659  decrease      decrease
5            -3.0   1.14622  1.14644  decrease          same
6            -1.0   1.14615  1.14632  decrease      decrease
7            -3.0   1.14589  1.14613      same      increase
8             2.0   1.14612  1.14611      same  